### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot
from sklearn import linear_model, ensemble, metrics, model_selection, feature_selection
from sklearn.ensemble import RandomForestClassifier

### Utilities Methods

In [2]:
def calculate_woe_iv(dataset, feature, target):
    # Calculate the total count of positive and negative outcomes
    total_pos = dataset[target].sum()
    total_neg = len(dataset) - total_pos

    # Calculate the WOE and IV for each category
    woe_iv_df = dataset.groupby(feature)[target].agg(['sum', 'count']).reset_index()
    woe_iv_df.columns = [feature, 'pos', 'total']
    woe_iv_df['neg'] = woe_iv_df['total'] - woe_iv_df['pos']
    woe_iv_df['pos_dist'] = woe_iv_df['pos'] / total_pos
    woe_iv_df['neg_dist'] = woe_iv_df['neg'] / total_neg
    woe_iv_df['woe'] = np.log(woe_iv_df['pos_dist'] / woe_iv_df['neg_dist'])
    woe_iv_df['iv'] = (woe_iv_df['pos_dist'] - woe_iv_df['neg_dist']) * woe_iv_df['woe']
    woe_iv_df = woe_iv_df.replace({'woe': {np.inf: 0, -np.inf: 0}})

    # Return the WOE and IV values
    return woe_iv_df[[feature, 'woe', 'iv']]

### Read the Fraud Data

In [3]:
transactions = pd.read_csv('transactions_raw.csv', sep=';')

# Convert 'trans_date_trans_time' to datetime format
transactions['trans_date_trans_time'] = pd.to_datetime(transactions['trans_date_trans_time'])

transactions['Year'] = transactions['trans_date_trans_time'].dt.year
transactions['Month'] = transactions['trans_date_trans_time'].dt.month
transactions['Day'] = transactions['trans_date_trans_time'].dt.day
transactions['Hour'] = transactions['trans_date_trans_time'].dt.hour
transactions['DayOfWeek'] = transactions['trans_date_trans_time'].dt.dayofweek

transactions = transactions.drop(['cc_num', 'first', 'last', 'street', 'unix_time', 'merchant', 'dob', 'merch_long', 'merch_lat',
                                  'lat', 'long'],
                                axis=1)

transactions.head()

C:\Users\Pathos-PC\AppData\Local\Temp\ipykernel_25296\1623075925.py:4: UserWarning:

Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,customer_id,trans_date_trans_time,category,amt,gender,city,state,zip,city_pop,job,trans_num,is_fraud,is_business_day,is_payment_day_5,is_payment_day_20,Year,Month,Day,Hour,DayOfWeek
0,309210,2019-05-29 23:37:00,shopping_net,793.81,M,Riverton,WY,82501.0,19408.0,"Lecturer, higher education",61c4bffa898c99f4fb0df460259f1b9e,1.0,1,0,0,2019,5,29,23,2
1,203887,2019-04-14 17:14:00,kids_pets,224.44,M,North Haverhill,NH,3774.0,1966.0,Structural engineer,8d71b2224f69eb526bae181c5200d05b,0.0,0,0,0,2019,4,14,17,6
2,250812,2019-05-05 13:47:00,personal_care,139.71,M,Cuyahoga Falls,OH,44223.0,47772.0,Equities trader,6e08872eb11ee3b4584e39b19bc72747,0.0,0,1,0,2019,5,5,13,6
3,73901,2019-02-12 20:52:00,kids_pets,74.98,F,Howes Cave,NY,12092.0,1304.0,Jewellery designer,7f2021868c3e1e304edca0354f8a4dc7,0.0,1,0,0,2019,2,12,20,1
4,233109,2019-04-27 20:35:00,misc_net,777.94,M,Birmingham,AL,35229.0,493806.0,Musician,fe86e6b60275e0043a6ce71d1cd7c74b,1.0,0,0,0,2019,4,27,20,5


### Exploratory Data Analysis

In [4]:
transactions.describe()

,customer_id,trans_date_trans_time,amt,zip,city_pop,is_fraud,is_business_day,is_payment_day_5,is_payment_day_20,Year,Month,Day,Hour,DayOfWeek
count,30000.000000,30000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,30000.0,30000.000000,30000.000000,30000.000000,30000.000000
mean,153430.646567,2019-03-21 07:44:14.916000,125.573572,48568.705767,8.760712e+04,0.130000,0.669767,0.030367,0.036667,2019.0,3.131400,15.883067,12.932033,3.023133
min,0.000000,2019-01-01 00:00:00,1.000000,1257.000000,2.300000e+01,0.000000,0.000000,0.000000,0.000000,2019.0,1.000000,1.000000,0.000000,0.000000
25%,75505.250000,2019-02-14 00:43:45,11.540000,25832.000000,7.410000e+02,0.000000,0.000000,0.000000,0.000000,2019.0,2.000000,8.000000,7.000000,1.000000
50%,151759.000000,2019-03-23 01:28:00,52.650000,48088.000000,2.463500e+03,0.000000,1.000000,0.000000,0.000000,2019.0,3.000000,16.000000,14.000000,3.000000
75%,231574.750000,2019-04-27 07:44:45,100.055000,71832.000000,2.112725e+04,0.000000,1.000000,0.000000,0.000000,2019.0,4.000000,24.000000,19.000000,5.000000
max,310239.000000,2019-05-30 12:45:00,6756.430000,99783.000000,2.906700e+06,1.000000,1.000000,1.000000,1.000000,2019.0,5.000000,31.000000,23.000000,6.000000
std,89503.905478,NaN,241.012148,26881.433146,2.928448e+05,0.336309,0.470305,0.171597,0.187945,0.0,1.389076,8.704730,7.267309,2.161717


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customer_id            30000 non-null  int64         
 1   trans_date_trans_time  30000 non-null  datetime64[ns]
 2   category               30000 non-null  object        
 3   amt                    30000 non-null  float64       
 4   gender                 30000 non-null  object        
 5   city                   30000 non-null  object        
 6   state                  30000 non-null  object        
 7   zip                    30000 non-null  float64       
 8   city_pop               30000 non-null  float64       
 9   job                    30000 non-null  object        
 10  trans_num              30000 non-null  object        
 11  is_fraud               30000 non-null  float64       
 12  is_business_day        30000 non-null  int64         
 13  i

### Transaction Fraud Distribution

In [6]:
# Calculate the proportions of fraud vs legitimate transactions
fraud_proportion = transactions['is_fraud'].value_counts(normalize=True).reset_index()
fraud_proportion.columns = ['Transaction Type', 'Percentage']
fraud_proportion['Transaction Type'] = fraud_proportion['Transaction Type'].map({0: 'Legitimate', 1: 'Fraudulent'})  # Mapping the types to string
fraud_proportion['Percentage'] *= 100  # Convert to percentage

# Create a bar chart
fig = px.bar(
    fraud_proportion, 
    x='Transaction Type', 
    y='Percentage',
    text='Percentage',
    title='Proportion of Fraudulent Transactions',
    color='Transaction Type',
    color_discrete_map={'Legitimate': 'green', 'Fraudulent': 'red'}  # Assigning colors
)

fig.update_layout(
    xaxis_title='Transaction Type',
    yaxis_title='Percentage',
    yaxis=dict(tickmode='array', tickvals=[0, 25, 50, 75, 100]), 
    showlegend=False
)

fig.update_traces(
    texttemplate='%{text:.2f}%',
    textposition='inside',
    insidetextanchor='middle' 
)

fig.update_layout(
    width=500, 
    height=400,
    margin=dict(l=15, r=15, t=50, b=20),
)

fig.show()


### Fraudulent Transactions by Merchant Category

In [7]:
# Calculate the proportions of fraud for each category.
category_proportions = transactions.groupby('category')['is_fraud'].value_counts(normalize=True).unstack() * 100
category_proportions.columns = ['Legitimate', 'Fraudulent']

# Convert the DataFrame to a long-form or tidy format
df_long = category_proportions.reset_index().melt(id_vars='category', value_vars=['Legitimate', 'Fraudulent'],
                                                  var_name='Transaction Type', value_name='Percentage')

# Define custom colors for legitimate and fraudulent transactions
custom_colors = {
    'Legitimate': 'lightblue',  # Change to any valid CSS color
    'Fraudulent': 'orange',  # Change to any valid CSS color
}

# Create a bar chart
fig = px.bar(
    df_long, 
    y='category', 
    x='Percentage', 
    color='Transaction Type', 
    orientation='h',
    text='Percentage',
    barmode='stack',
    color_discrete_map={'Legitimate': '#ccc', 'Fraudulent': 'crimson'},  
    title='Percentage of Fraudulent vs Legitimate Transactions by Merchant Category'
)

fig.update_layout(
    xaxis=dict(title='Percentage of Transactions', range=[0, 100]),
    yaxis=dict(title='Merchant Category'),
    legend_title_text='Transaction Type',
    uniformtext_minsize=7, 
    uniformtext_mode='hide',
)

# Update the traces to adjust text position and set text inside the bars
for trace in fig.data:
    trace.texttemplate = '%{text:.1f}%'
    trace.textposition = 'inside'
    trace.insidetextanchor = 'middle'

fig.update_layout(
    width=1300,
    height=500, 
    margin=dict(l=15, r=15, t=50, b=20),
)

fig.show()


### Fraudulent Transactions by BDay

In [8]:
# Calculate the proportions of fraud for each BDay
category_proportions = transactions.groupby('is_business_day')['is_fraud'].value_counts(normalize=True).unstack() * 100
category_proportions.columns = ['Legitimate', 'Fraudulent']

# Convert the DataFrame to a long-form or tidy format
df_long = category_proportions.reset_index().melt(id_vars='is_business_day', value_vars=['Legitimate', 'Fraudulent'],
                                                  var_name='Transaction Type', value_name='Percentage')

# Create a bar chart
fig = px.bar(
    df_long, 
    y='is_business_day', 
    x='Percentage', 
    color='Transaction Type', 
    orientation='h',
    text='Percentage',
    barmode='stack',
    color_discrete_map={'Legitimate': '#ccc', 'Fraudulent': 'crimson'},
    title='Fraudulent vs Legitimate Transactions by BDay'
)

fig.update_layout(
    yaxis=dict(title='Is Business Day'),
    xaxis=dict(title='Percentage of Transactions', range=[0, 100]),
    legend_title_text='Transaction Type',
    uniformtext_minsize=7, 
    uniformtext_mode='hide',
)

# Update the traces to adjust text position
for trace in fig.data:
    trace.texttemplate = '%{text:.1f}% <br> out Total '
    trace.textposition = 'inside'

fig.update_layout(
    width=1000,
    height=400,
    margin=dict(l=15, r=15, t=50, b=20),
)

fig.show()


### Fraudulent Transactions Over Hours in a Day

In [9]:
# transactions['transaction_date'] = pd.to_datetime(transactions['trans_date_trans_time']).dt.date
transactions_per_hour_fraud = transactions[transactions['is_fraud'] == 1].groupby('Hour').size()

# Convert your groupby object to a DataFrame
df_transactions_per_hour_fraud = transactions_per_hour_fraud.reset_index()
df_transactions_per_hour_fraud.columns = ['Hour', 'Number of Fraudulent Transactions']

# Create the plot
fig = px.area(
    df_transactions_per_hour_fraud, 
    x='Hour', 
    y='Number of Fraudulent Transactions', 
    title='Fraudulent Transactions Over Hours in a Day',
    color_discrete_sequence=['crimson'],
    labels={'Hour': 'Hour of the Day', 'Number of Fraudulent Transactions': 'Number of Transactions'}
)

fig.update_layout(
    width=1200,
    height=400,
    margin=dict(l=15, r=15, t=50, b=20),
)

fig.show()

In [10]:
# Your existing code to prepare the data
transactions_per_hour_fraud = transactions[transactions['is_fraud'] == 1].groupby('Day').size()
transactions_per_hour_non_fraud = transactions[transactions['is_fraud'] == 0].groupby('Day').size()

# Convert your groupby object to a DataFrame and rename the columns appropriately
df_transactions_per_hour_fraud = transactions_per_hour_fraud.reset_index()
df_transactions_per_hour_fraud.columns = ['Date', 'Number of Fraudulent Transactions']

df_transactions_per_hour_non_fraud = transactions_per_hour_non_fraud.reset_index()
df_transactions_per_hour_non_fraud.columns = ['Date', 'Number of Non-Fraudulent Transactions']

# Calculate the means
mean_fraudulent_transactions = df_transactions_per_hour_fraud['Number of Fraudulent Transactions'].mean()
mean_non_fraudulent_transactions = df_transactions_per_hour_non_fraud['Number of Non-Fraudulent Transactions'].mean()

# Create traces for the plot
trace2 = go.Scatter(
    x=df_transactions_per_hour_fraud['Date'],
    y=df_transactions_per_hour_fraud['Number of Fraudulent Transactions'],
    mode='lines',
    name='Fraudulent Transactions',
    line=dict(color='crimson', width=1),
    fill='tozeroy',
    hovertemplate=f"Day: %{{x}}<br>Amount of Transactions: %{{y}}<br>Mean: {mean_fraudulent_transactions:.2f}<extra></extra>"
)

trace1 = go.Scatter(
    x=df_transactions_per_hour_non_fraud['Date'],
    y=df_transactions_per_hour_non_fraud['Number of Non-Fraudulent Transactions'],
    mode='lines',
    name='Non-Fraudulent Transactions',
    line=dict(color='lightgreen', width=1),
    fill='tozeroy',
    hovertemplate=f"Day: %{{x}}<br>Amount of Transactions: %{{y}}<br>Mean: {mean_non_fraudulent_transactions:.2f}<extra></extra>"
)

# Combine the traces
data = [trace1, trace2]

layout = go.Layout(
    title='Transaction Frequency Over Days in a Month',
    xaxis=dict(title='Day of the Month'),
    yaxis=dict(title='Number of Transactions'),
    hovermode='closest'
)

fig = go.Figure(data=data, layout=layout)

# Loop through the dates and add vertical dashed lines
for date in df_transactions_per_hour_fraud['Date']:
    fig.add_shape(type="line",
                  x0=date, y0=0, x1=date, y1=1,
                  xref="x", yref="paper",
                  line=dict(color="#666", width=0.5, dash="dot"))
    
fig.update_layout(
    width=1200, 
    height=400, 
    margin=dict(l=15, r=15, t=50, b=20),
)

iplot(fig)


In [11]:
# Calculate transactions per state
transactions_per_state = transactions.groupby('state').size().sort_values(ascending=False).reset_index(name='Transactions')

# Function to create traces for top N states
def create_trace_for_top_n(n):
    top_states = transactions_per_state.head(n)
    return go.Bar(
        x=top_states['state'],
        y=top_states['Transactions'],
        marker=dict(color=top_states['Transactions'], coloraxis="coloraxis")
    )

# Create the initial figure with the top 10 states
fig = go.Figure(data=create_trace_for_top_n(10))

# Update the layout to add Dropdown menu functionality
fig.update_layout(
    title='Number of Transactions by State',
    xaxis=dict(title='State'),
    yaxis=dict(title='Number of Transactions'),
    coloraxis=dict(colorscale='Fall', colorbar_title='Transactions'),
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'y': [transactions_per_state.head(10)['Transactions']],
                           'x': [transactions_per_state.head(10)['state']],
                           'type': 'bar'}],
                    label='Top 10',
                    method='restyle'
                ),
                dict(
                    args=[{'y': [transactions_per_state.head(20)['Transactions']],
                           'x': [transactions_per_state.head(20)['state']],
                           'type': 'bar'}],
                    label='Top 20',
                    method='restyle'
                ),
                dict(
                    args=[{'y': [transactions_per_state.head(30)['Transactions']],
                           'x': [transactions_per_state.head(30)['state']],
                           'type': 'bar'}],
                    label='Top 30',
                    method='restyle'
                )
            ]),
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=-0.05,
            xanchor='left',
            y=1.23,
            yanchor='top'
        ),
    ],
    width=900,
    height=400,
)

fig.show()


### Feature Composition and Encoder

Was identified that the creation of some features could help the prediction power of our model. <br>
Also, we need to convert categorical data. Was used an encoder based on the Weight of Evidence method.

In [12]:
# Create features indicating whether it's hour 23, hour 1, hour 2, or hour 3
transactions['is_hour_23'] = (transactions['Hour'] == 23).astype(int)
transactions['is_hour_1'] = (transactions['Hour'] == 1).astype(int)
transactions['is_hour_2'] = (transactions['Hour'] == 2).astype(int)
transactions['is_hour_3'] = (transactions['Hour'] == 3).astype(int)

# Create features indicating whether it's day 8, 12, 20 and 25
transactions['is_day_8'] = (transactions['Day'] == 8).astype(int)
transactions['is_day_12'] = (transactions['Day'] == 12).astype(int)
transactions['is_day_20'] = (transactions['Day'] == 20).astype(int)
transactions['is_day_25'] = (transactions['Day'] == 25).astype(int)

In [13]:
# Calculate WOE and IV for the specified columns
woe_iv_category = calculate_woe_iv(transactions, 'category', 'is_fraud')
woe_iv_gender = calculate_woe_iv(transactions, 'gender', 'is_fraud')
woe_iv_state = calculate_woe_iv(transactions, 'state', 'is_fraud')
woe_iv_city = calculate_woe_iv(transactions, 'city', 'is_fraud')
woe_iv_job = calculate_woe_iv(transactions, 'job', 'is_fraud')

# Create the dictionaries
category_woe_dict = woe_iv_category.set_index('category')['woe'].to_dict()
gender_woe_dict = woe_iv_gender.set_index('gender')['woe'].to_dict()
state_woe_dict = woe_iv_state.set_index('state')['woe'].to_dict()
city_woe_dict = woe_iv_city.set_index('city')['woe'].to_dict()
job_woe_dict = woe_iv_job.set_index('job')['woe'].to_dict()

# Replace the column in the original dataframe with the WoE values
transactions['category'] = transactions['category'].replace(category_woe_dict)
transactions['gender'] = transactions['gender'].replace(gender_woe_dict)
transactions['state'] = transactions['state'].replace(state_woe_dict)
transactions['city'] = transactions['city'].replace(city_woe_dict)
transactions['job'] = transactions['job'].replace(job_woe_dict)

c:\Users\Pathos-PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

c:\Users\Pathos-PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

c:\Users\Pathos-PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

C:\Users\Pathos-PC\AppData\Local\Temp\ipykernel_25296\488375401.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\Pathos-PC\AppData\Local\Temp\ipykernel_25296\488375401.py:17: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version

### Feature Relevance Discovery

In order to discover the most relevant features to our model was used a tree-based method (Random Forest). <br>

In [14]:
X = transactions.drop(['trans_date_trans_time', 'is_fraud', 'trans_num', 'job'], axis=1)
y = transactions.is_fraud

# Fit a Random Forest model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X, y)

RandomForestClassifier(n_estimators=200, random_state=42)

In [15]:
# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for the feature importances
importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance and select the top 20 features
importances_df = importances_df.sort_values(by='Importance', ascending=False).head(20)


In [16]:
# Create a bar chart
fig = px.bar(
    importances_df,
    x='Feature',
    y='Importance',
    title='Top 20 Feature Importances (Random Forest)',
    labels={'Importance': 'Gini Importance'},
    color='Importance',
    color_continuous_scale=['#F1BF98', '#F1A073', '#EE8258', '#EB643D', '#E84522'],
)

fig.update_layout(
    width=1200, 
    height=900, 
    margin=dict(l=15, r=15, t=50, b=20),
)

fig.show()

In [33]:
relevant_features = importances_df.Feature.to_list() + ['is_fraud']

In [35]:
transactions[relevant_features].to_csv('transactions_processed.csv')